In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import gc
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

SAMPLE_SIZE = 0.50
#Caso con edad en mean en vez de 200 para los nan unicamente
#nuevos features
#denominacion empresa mean encoding

%matplotlib inline

In [2]:
avisos = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisosONEHOT.csv")
postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulaciones.csv")
postulantes = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulantesONEHOT.csv")

avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos_normales = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisos.csv")
avisos_normales = avisos_normales.loc[:,["nombre_area","idaviso","tipo_de_trabajo","nivel_laboral"]]

vistas = pd.read_csv("Data/fiuba_entrenamiento/pocho/vistas.csv")

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [3]:
avisos.denominacion_empresa

0                                   VENTOR
1                      Wurth Argentina S.A
2                       ELECTRO OUTLET SRL
3                      Wurth Argentina S.A
4                      Wurth Argentina S.A
5                      Wurth Argentina S.A
6        Agencia Oficial Alejandro Arizaga
7                                cerantola
8                        Hochschild Mining
9                               YAMANIL SA
10                   Adecco -Región Office
11                Romasanta y Asociados SA
12                          Bairestron SRL
13                          Directway S.A.
14                     Wurth Argentina S.A
15           SC Johnson & Son de Argentina
16                          Unión Personal
17                            Motomel S.A.
18                          Asoko Tempo SA
19                         TELEDIFUSORA SA
20                         RE/MAX TITANIUM
21                    Grupo Pujol Figueras
22                                  VENTOR
23         

In [4]:
vistas['cantidad']=0
vistas2=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,1
1,48375,RwVdKR,1
2,169730,1KjXB,1
3,169730,2AKzxa,1
4,169730,6LJ64,1


In [5]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas.head()

,idpostulante,cantidad
0,0002q,3
1,0005E,25
2,000R8,4
3,001XE,12
4,003k9,26


In [6]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,1,20
1,48375,RwVdKR,1,10
2,169730,1KjXB,1,34
3,169730,2AKzxa,1,1
4,169730,6LJ64,1,10


In [7]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.050000
1,48375,RwVdKR,0.100000
2,169730,1KjXB,0.029412
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.100000


In [8]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})
vistas_final = pd.merge(vistas3,avisos_normales, on= "idaviso", how = "left")
vistas_final.head()

,idaviso,idpostulante,cantidad,nombre_area,tipo_de_trabajo,nivel_laboral
0,18,BolNL,0.050000,NaN,NaN,NaN
1,48375,RwVdKR,0.100000,NaN,NaN,NaN
2,169730,1KjXB,0.029412,NaN,NaN,NaN
3,169730,2AKzxa,1.000000,NaN,NaN,NaN
4,169730,6LJ64,0.100000,NaN,NaN,NaN


In [9]:
no_postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/azar_total.csv")
postulaciones=postulaciones.append(no_postulaciones.sample(len(postulaciones)*3//4), ignore_index=True)
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
postulaciones = postulaciones.sample(int(round(len(postulaciones)*SAMPLE_SIZE)))
no_postulaciones = 0
gc.collect()

C:\Users\Pocho\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


71

In [10]:
vistas_final["vistas_por_area"]=0
vistas_final["vistas_por_tipo"]=0
vistas_final["vistas_por_nivel"]=0

vistas_area = vistas_final.groupby(['idpostulante','nombre_area'],as_index=False).agg({'vistas_por_area':'count'})
vistas_tipo= vistas_final.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'vistas_por_tipo':'count'})
vistas_nivel=vistas_final.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'vistas_por_nivel':'count'})

vistas_final = 0
gc.collect()

7

In [11]:
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [12]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones = pd.merge(postulaciones,avisos_normales, on = "idaviso", how = "left")
postulaciones.drop("nombre_area_y", axis = 1, inplace=True)
postulaciones.rename({"nombre_area_x":"nombre_area"},axis=1,inplace=True)
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['vistas_por_area']=postulaciones['vistas_por_area'].fillna(0)
postulaciones['vistas_por_tipo']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones['vistas_por_nivel']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones["vistas_al_aviso"] = postulaciones["vistas_al_aviso"].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)
postulaciones.cantidad = postulaciones.cantidad.fillna(0)
postulaciones.vistas_postulante = postulaciones.vistas_postulante.fillna(0)

postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio
1760403,1112348618,6r5rD6L,1,29.0,True,False,False,False,False,False,...,False,Full-time,Junior,23.0,0.0,0.000000,0.0,14.0,14.0,False
5498726,1111839019,5WWEYN,0,34.0,False,True,False,False,False,False,...,True,Part-time,Senior / Semi-Senior,5.0,0.0,0.000000,0.0,1.0,1.0,False
1799961,1112287852,KBdXPeq,1,20.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,41.0,0.0,0.000000,0.0,31.0,31.0,False
1498300,1112368151,ak4KJ1m,0,27.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,5.0,0.0,0.000000,0.0,4.0,4.0,False
3969608,1112365274,ekOk0E2,0,28.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,0.0,0.0,0.000000,0.0,0.0,0.0,False
2634790,1112346591,JBrNBOO,1,22.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,7.0,0.0,0.000000,0.0,6.0,6.0,False
257266,1112398441,Dr628GW,0,30.0,False,True,False,False,False,False,...,False,Full-time,Junior,0.0,0.0,0.000000,0.0,0.0,0.0,False
540921,1112293428,5mPJPjq,0,42.0,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,0.0,0.0,0.000000,0.0,0.0,0.0,False
98034,1112177605,xkd6dNK,0,34.0,False,True,False,False,False,False,...,False,Full-time,Jefe / Supervisor / Responsable,33.0,0.0,0.000000,0.0,31.0,31.0,False
3155809,1112456642,NzPbA24,1,27.0,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,17.0,1.0,0.058824,1.0,16.0,16.0,True


In [13]:
postulaciones_area = postulaciones.groupby(['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_tipo= postulaciones.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_nivel =postulaciones.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_area=postulaciones_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_tipo=postulaciones_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_nivel=postulaciones_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})

In [14]:
postulaciones.dropna(axis = 1, inplace=True)

In [15]:
postulaciones_area.head()

,idpostulante,nombre_area,postulaciones_misma_area
0,0z5Dmrd,Logística,1
1,0z5Dmrd,Recursos Humanos,2
2,0z5Dmrd,Tecnologia / Sistemas,1
3,0z5Dmrd,Ventas,2
4,0z5JW1r,Administración de Personal,1


In [16]:
postulaciones=pd.merge(postulaciones,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')

postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones.drop(["tipo_de_trabajo","nivel_laboral"], axis = 1, inplace=True)



In [20]:
#mean encoding
area_mean = postulaciones.groupby("nombre_area").agg({"sepostulo":"mean"})
area_mean.head()

means = area_mean.to_dict()["sepostulo"]
postulaciones.nombre_area = postulaciones.nombre_area.replace(means)
postulaciones.sample(10)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
5216298,1112312753,bOzm4pE,0,26.0,True,False,False,False,False,False,...,9.0,0.0,0.000000,0.0,7.0,7.0,False,0,3,2
5568696,1112494573,BmDaZza,0,29.0,True,False,False,False,False,False,...,0.0,0.0,0.000000,0.0,0.0,0.0,False,1,9,6
2307343,1112278041,EzpQYbo,0,23.0,False,True,False,False,False,False,...,41.0,0.0,0.000000,0.0,40.0,40.0,False,0,35,4
5283416,1112207782,JBr2REO,0,35.0,True,False,False,False,False,False,...,2.0,0.0,0.000000,0.0,1.0,1.0,False,0,11,7
2502430,1112406262,PmGebqv,1,36.0,False,True,False,False,False,False,...,5.0,1.0,0.200000,3.0,5.0,5.0,True,6,12,8
1336506,1112428767,YABVdD,1,50.0,False,True,False,False,False,False,...,33.0,1.0,0.030303,13.0,28.0,28.0,True,1,12,8
5481269,1112511376,dYjNzpA,0,30.0,True,False,False,False,False,False,...,2.0,0.0,0.000000,0.0,1.0,1.0,False,0,5,3
2237660,1112303922,pz9M0LZ,1,25.0,False,True,False,False,False,False,...,18.0,0.0,0.000000,2.0,14.0,14.0,False,2,18,16
4254956,1112353927,Npw5wM,1,40.0,False,True,False,False,False,False,...,14.0,1.0,0.071429,13.0,14.0,14.0,True,16,26,17
4561658,1112424885,eDpLbj,1,40.0,False,True,False,False,False,False,...,14.0,1.0,0.071429,9.0,11.0,11.0,True,2,5,4


In [21]:
denom_mean = postulaciones.groupby("denominacion_empresa").agg({"sepostulo":"mean"})
means2 = denom_mean.to_dict()["sepostulo"]
print("start replace")
postulaciones.denominacion_empresa = postulaciones.denominacion_empresa.replace(means2)
#postulaciones.sample(10)

start replace


In [22]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
#features.remove('denominacion_empresa')
#features.remove("ciudad")
#features.remove("mapacalle")

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

postulaciones=0

gc.collect()

Train:  5012211 Test:  556913


83

In [20]:
#model = XGBClassifier(objective='binary:logistic',max_depth=18,learning_rate=0.1,subsample=0.8,colsample_bytree=0.8,n_estimators=1200,silent=False,n_jobs=-1,verbose=1)
#dt=model.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=5)


[0]	validation_0-error:0.10226
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.097105
[2]	validation_0-error:0.095076
[3]	validation_0-error:0.093445
[4]	validation_0-error:0.092377
[5]	validation_0-error:0.091694
[6]	validation_0-error:0.091071
[7]	validation_0-error:0.090455
[8]	validation_0-error:0.089997
[9]	validation_0-error:0.088719
[10]	validation_0-error:0.088561
[11]	validation_0-error:0.088117
[12]	validation_0-error:0.08802
[13]	validation_0-error:0.087354
[14]	validation_0-error:0.087087
[15]	validation_0-error:0.086368
[16]	validation_0-error:0.086198
[17]	validation_0-error:0.086129
[18]	validation_0-error:0.085776
[19]	validation_0-error:0.085488
[20]	validation_0-error:0.085241
[21]	validation_0-error:0.084838
[22]	validation_0-error:0.084497
[23]	validation_0-error:0.084172
[24]	validation_0-error:0.083542
[25]	validation_0-error:0.0831
[26]	validation_0-error:0.082834
[27]	validation_0-error:0.082484
[28]	validation_0-error:0

In [18]:
model = joblib.load("Data/fiuba_entrenamiento/pocho/modelos/XGBoostMean6-BEST.pkl")

In [23]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=model.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9803640005694283


In [23]:
#joblib.dump(model, "Data/fiuba_entrenamiento/pocho/modelos/XGBoostMean6.pkl")

['Data/fiuba_entrenamiento/pocho/modelos/XGBoostMean6.pkl']

In [24]:
gc.collect()

0

In [ ]:
prediccion = pd.read_csv("Data/fiuba_entrenamiento/test_final_100k.csv")
prediccion = pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
prediccion = pd.merge(prediccion,avisos,on='idaviso',how='inner')
prediccion = pd.merge(prediccion,avisos_normales,on='idaviso',how='inner')
prediccion.drop("nombre_area_y", axis = 1, inplace=True)
prediccion.rename({"nombre_area_x":"nombre_area"},axis=1,inplace=True)

prediccion = pd.merge(prediccion,vistas,on='idpostulante',how='left')
prediccion.sample(10)


prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion['cantidad']=prediccion['cantidad'].fillna(0)
prediccion['vistas_postulante']=prediccion['vistas_postulante'].fillna(0)
prediccion['vistas_al_aviso']=prediccion['vistas_al_aviso'].fillna(0)
prediccion['lo_vio']=(prediccion['vistas_al_aviso']>0)

prediccion=pd.merge(prediccion,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion=pd.merge(prediccion,vistas_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['vistas_por_area']=prediccion['vistas_por_area'].fillna(0)
prediccion['vistas_por_tipo']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['vistas_por_nivel']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0)
prediccion.nombre_area = prediccion.nombre_area.replace(means)
prediccion.denominacion_empresa = prediccion.denominacion_empresa.replace(means2)
prediccion.sample(10)

In [25]:
prediccion.head()

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,26.0,0.0,0.0,False,0.0,3.0,0.0,3.0,20.0,20.0
1,1,739260,6v1xdL,31.0,False,True,False,False,False,False,...,4.0,0.0,0.0,False,2.0,32.0,0.0,0.0,3.0,3.0
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,10.0,0.0,0.0,False,0.0,4.0,0.0,1.0,6.0,6.0
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,0.0,0.0,0.0,False,0.0,5.0,1.0,0.0,0.0,0.0
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,1.0,0.0,0.0,False,0.0,4.0,0.0,0.0,1.0,1.0


In [26]:
x_final=prediccion[features]
y_final=model.predict_proba(x_final)
y_final2=[]
print(model.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

prediccion['sepostulo']=y_final2
prediccion.head()


[0 1]
100000
100000


,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel,sepostulo
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,0.0,0.0,False,0.0,3.0,0.0,3.0,20.0,20.0,0.010868
1,1,739260,6v1xdL,31.0,False,True,False,False,False,False,...,0.0,0.0,False,2.0,32.0,0.0,0.0,3.0,3.0,0.054804
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,0.0,0.0,False,0.0,4.0,0.0,1.0,6.0,6.0,0.005120
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,0.0,0.0,False,0.0,5.0,1.0,0.0,0.0,0.0,0.004027
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,0.0,0.0,False,0.0,4.0,0.0,0.0,1.0,1.0,0.000871


In [27]:
prediccion=prediccion[['id','sepostulo']]
print(prediccion.sepostulo.mean())
prediccion.head()

0.49345589480857555


,id,sepostulo
0,0,0.010868
1,1,0.054804
2,2,0.005120
3,3,0.004027
4,4,0.000871


In [28]:
prediccion.to_csv("Data/fiuba_entrenamiento/pocho/predicciones/xgbmean6.csv",index=False)